In [1]:
#imports 
import json
import pandas as pd
import numpy as np
import torch
from torch import nn
import warnings
import re
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [2]:
from google.colab import drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Arreglar los datos
df = pd.read_csv("/content/drive/MyDrive/Datathon/metricbeabt-so-28042023.csv")

Importante elegir el servidor en este punto

In [5]:
#Guardar los datos por evento
df_event_memory = df[df["event"] == "{'dataset': 'system.memory'}"][df["host"] == "{'name': 'PRODUCCION'}"]
df_event_cpu = df[df["event"] == "{'dataset': 'system.cpu'}"][df["host"] == "{'name': 'PRODUCCION'}"]
df_event_network = df[df["event"] == "{'dataset': 'system.network'}"][df["host"] == "{'name': 'PRODUCCION'}"]
df_event_memory = df_event_memory.dropna()
df_event_cpu = df_event_cpu.dropna()
df_event_network = df_event_network.dropna()

<ipython-input-5-a21875df7088>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_event_memory = df[df["event"] == "{'dataset': 'system.memory'}"][df["host"] == "{'name': 'PRODUCCION'}"]
<ipython-input-5-a21875df7088>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_event_cpu = df[df["event"] == "{'dataset': 'system.cpu'}"][df["host"] == "{'name': 'PRODUCCION'}"]
<ipython-input-5-a21875df7088>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_event_network = df[df["event"] == "{'dataset': 'system.network'}"][df["host"] == "{'name': 'PRODUCCION'}"]


In [6]:
##Ordenamos segun timestamp
df_event_memory = df_event_memory.sort_values(by=['@timestamp'])
df_event_cpu = df_event_cpu.sort_values(by=['@timestamp'])
df_event_network = df_event_network.sort_values(by=['@timestamp'])

# CPU

In [7]:
x = df_event_cpu["system"]
x = x.dropna()
df_cpu = pd.DataFrame()
df_cpu["Idle"] = []
df_cpu["System"] = []
df_cpu["User"] = []
df_cpu["Core"] = []

In [8]:
u = 0

for row in x:
    row = row.replace(" ", "")
    row = row.replace("{", "")
    row = row.replace("}", "")
    row = row.replace("'", "")
    row = row.split(",")
    row = [i.split(":") for i in row]
    rows = [[row[1][2], row[3][2], row[2][2], row[0][2]]]
    df_cpu.loc[u] = rows[0]
    u += 1
    if u%10 == 0:
      print(u)
df_cpu


10
20
30
40
50
60
70
80
90
100


,Idle,System,User,Core
0,7.7042,0.194,0.1018,8
1,7.7129,0.19190000000000002,0.0952,8
2,7.8359000000000005,0.1203,0.0438,8
3,7.7512,0.1549,0.09380000000000001,8
4,7.7875,0.1531,0.0594,8
...,...,...,...,...
95,7.5954,0.23120000000000002,0.1734,8
96,7.7762,0.17070000000000002,0.053200000000000004,8
97,7.6943,0.1762,0.1295,8
98,7.7718,0.1563,0.0719,8


# Memory

In [10]:
y = df_event_memory["system"]
y = y.dropna()
df_memory = pd.DataFrame()
df_memory["Actpercen"] = []

In [13]:
u = 0

for row in y:
    row = row.replace(" ", "")
    row = row.replace("{", "")
    row = row.replace("}", "")
    row = row.replace("'", "")
    row = row.split(",")
    row = [i.split(":") for i in row]
    rows = [[row[3][1]]]
    df_memory.loc[u] = rows[0]
    u += 1
    if u%50000 == 0:
      print(u)
df_memory

,Actpercen
0,0.6164000000000001
1,0.6161
2,0.6161
3,0.6162000000000001
4,0.616
...,...
95,0.6136
96,0.6134000000000001
97,0.6141
98,0.6142000000000001


# Network

In [14]:
z = df_event_network["system"]
z = z.dropna()
df_network = pd.DataFrame()
df_network["in"] = []
df_network["out"] = []

In [15]:
u = 0

for row in z:
    row = row.replace(" ", "")
    row = row.replace("{", "")
    row = row.replace("}", "")
    row = row.replace("'", "")
    row = row.split(",")
    row = [i.split(":") for i in row]
    rows = [[row[0][3], row[1][2]]]
    df_network.loc[u] = rows[0]
    u += 1
    if u%50000 == 0:
      print(u)
df_network

,in,out
0,780698195403,137473086217
1,0,0
2,780702831269,137474112038
3,0,0
4,0,0
...,...,...
95,0,0
96,0,0
97,780906113947,137497322457
98,780909545727,137497431472


In [ ]:
df_cpu.to_csv("/content/drive/MyDrive/Datathon/CPU.csv")
df_network.to_csv("/content/drive/MyDrive/Datathon/network.csv")
df_memory.to_csv("/content/drive/MyDrive/Datathon/memory.csv")